In [47]:
import fasttext
import pandas as pd
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Jason\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [3]:
# Get the fasttext model (we are using the largest one they offer [600B tokens])
fasttext_model = fasttext.load_model('models/crawl-300d-2M-subword.bin')

In [33]:
# Get the dataset of computer parts
computers_df = pd.read_json('data/computers_train/computers_train_xlarge_normalized.json.gz',compression='gzip', lines=True)

In [34]:
# The size of our training set
len(computers_df)

68461

In [38]:
# Just see that data that we are working with
computers_df.head()

,id_left,title_left,description_left,brand_left,price_left,specTableContent_left,keyValuePairs_left,category_left,cluster_id_left,identifiers_left,...,description_right,brand_right,price_right,specTableContent_right,keyValuePairs_right,category_right,cluster_id_right,identifiers_right,label,pair_id
0,5490217,hp intel xeon x5560 prijzen tweakers,None,None,None,categorie processors merk hp product hp intel ...,"{'categorie': 'processors', 'merk': 'hp', 'pro...",Computers_and_Accessories,1679624,"[{'/mpn': '[495906b21]'}, {'/gtin13': '[884420...",...,description intel xeon x5560 ml350 g6 2 80ghz ...,hp enterprise,usd 213 85,specifications category proliant processor sub...,"{'category': 'proliant processor', 'sub catego...",Computers_and_Accessories,1679624,[{'/mpn': '[495906b21]'}],1,5490217#16248399
1,16876009,"495906 b21 hp x5560 2 80ghz ml350 g6 , null ne...",description intel xeon x5560 ml350 g6 2 80ghz ...,hp enterprise,None,specifications category proliant processor sub...,"{'category': 'proliant processor', 'sub catego...",Computers_and_Accessories,1679624,"[{'/sku': '[495906b21]'}, {'/mpn': '[495906b21...",...,description intel xeon x5560 ml350 g6 2 80ghz ...,hp enterprise,usd 213 85,specifications category proliant processor sub...,"{'category': 'proliant processor', 'sub catego...",Computers_and_Accessories,1679624,[{'/mpn': '[495906b21]'}],1,16876009#16248399
2,16543750,asus motherboard lga2066 ddr4 m 2 u atx 2xgbe ...,None,None,None,None,None,Computers_and_Accessories,109916,"[{'/sku': '[34181655]'}, {'/mpn': '[primex299d...",...,placa base atx socket lga2066 chipset intel x2...,None,None,None,None,Computers_and_Accessories,109916,[{'/productID': '[90mb0ty0m0eay0]'}],1,16543750#14031864
3,16721450,asus prime x299 deluxe prijzen tweakers,None,None,None,categorie moederborden merk asus product asus ...,"{'categorie': 'moederborden', 'merk': 'asus', ...",Computers_and_Accessories,109916,"[{'/mpn': '[primex299deluxe, 90mb0ty0m0eay0]'...",...,support for x series intel core processors sli...,asus,None,None,None,Computers_and_Accessories,109916,"[{'/mpn': '[90mb0ty0m0eay0]'}, {'/gtin13': '[4...",1,16721450#10358026
4,14031864,asus prime x299 deluxe,placa base atx socket lga2066 chipset intel x2...,None,None,None,None,Computers_and_Accessories,109916,[{'/productID': '[90mb0ty0m0eay0]'}],...,atx quad channel ddr4 3 x pcie 3 0 x16 2 x m 2...,asus,None,None,None,Computers_and_Accessories,109916,"[{'/productID': '[asux29del]'}, {'/mpn': '[pri...",1,14031864#4588573


In [105]:
to_stop = stopwords.words('english')
punctuation = "!”#$%&’()*+,-./:;<=>?@[\]^_`{|}~ "
for c in punctuation:
    to_stop.append(c)

to_stop.append('null')
print(to_stop)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [135]:
# Organizing and normalizing the data
"""
Essentially, we want to only have three attributes for each training example: title_one, title_two, label
For normalization, we are just going to use the nltk stopwords and punctuation
"""

def preprocessing():
    column_names = ['title_one', 'title_two', 'label']
    norm_computers = pd.DataFrame(columns = column_names)
    for row in computers_df.itertuples():
        title_left = row.title_left.split(' ')
        title_right = row.title_right.split(' ')
        
        temp_title_left = []
        for word in title_left:
            append_word = True
            for stop in to_stop:
                if (word == stop):
                    append_word = False
            if append_word:
                temp_title_left.append(word)
                    
        
        temp_title_right = []
        for word in title_right:
            append_word = True
            for stop in to_stop:
                if (word == stop):
                    append_word = False
            if append_word:
                temp_title_right.append(word)

        title_left = ' '.join(temp_title_left)
        title_right = ' '.join(temp_title_right)
        
        norm_computers = norm_computers.append(pd.DataFrame([[title_left, title_right, row.label]], columns=column_names))
        
    return norm_computers
        

In [136]:
norm_computers = preprocessing()

In [137]:
len(norm_computers)

68461

In [139]:
norm_computers.tail()

,title_one,title_two,label
0,seagate barracuda es 2 st3750330ns 750gb 3 5 h...,454141 002 hp 500 gb 7 2k 3 5 sata 10 pack who...,0
0,seagate barracuda es 2 750gb sata ii 32mb cach...,782995 001 hp 6 tb 6g 7 2k 3 5 sas hdd new who...,0
0,st3750330ns seagate 750 gb 15k 3 5 3g sata,seagate desktop hdd 1tb sata 6gb,0
0,st3750330ns seagate 750 gb 15k 3 5 3g sata,454141 002 hp 500 gb 7 2k 3 5 sata 2 pack whol...,0
0,seagate barracuda es 2 st3750330ns 750gb 3 5 h...,454141 002 hp 500 gb 7 2k 3 5 sata 10 pack who...,0


In [140]:
norm_computers.to_csv('data/computers_train/computers_train_xlarge_norm_simple.csv', index=False)